# How should I price auto insurance in the United States?

## Introduction

**Business Context.** The ability to price an insurance quote properly has a significant impact on insurers' management decisions and financial statements. You are the chief data scientist at a new startup insurance company focusing on providing affordable insurance to millennials. You are tasked to assess the current state of insurance companies to see what factors large insurance providers charge premiums for. Fortunately for you, your company has compiled a dataset by surveying what people currently pay for insurance from large companies. Your findings will be used as the basis of developing your company's millenial car insurance offering. 

**Business Problem.** Your task is to build a **minimal** model to predict the cost of insurance from the data set using various characteristics of a policyholder.

**Analytical Context.** The data resides in a CSV file which has been pre-cleaned for you and can directly be read in. Throughout the case, you will be iterating on your initial model many times based on common pitfalls that arise which we discussed in previous cases. You will be using the Python `statsmodels` package to create and analyze these linear models.

In [1]:
! pip install -r requirements.txt

     |████████████████████████████████| 28.4 MB 77 kB/s  eta 0:00:01|█████▊                          | 5.1 MB 4.2 MB/s eta 0:00:06K     |███████████▎                    | 10.0 MB 4.2 MB/s eta 0:00:05 |█████████████▎                  | 11.8 MB 4.2 MB/s eta 0:00:04
  Using cached seaborn-0.11.1-py3-none-any.whl (285 kB)
  Using cached statsmodels-0.12.2-cp38-cp38-manylinux1_x86_64.whl (9.4 MB)
  Using cached Pillow-8.3.1-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.whl (3.0 MB)
  Using cached matplotlib-3.4.2-cp38-cp38-manylinux1_x86_64.whl (10.3 MB)
  Using cached kiwisolver-1.3.1-cp38-cp38-manylinux1_x86_64.whl (1.2 MB)
  Using cached cycler-0.10.0-py2.py3-none-any.whl (6.5 kB)
  Using cached patsy-0.5.1-py2.py3-none-any.whl (231 kB)


In [2]:
### Load relevant packages

import pandas                  as pd
import numpy                   as np
import matplotlib.pyplot       as plt
import seaborn                 as sns
import statsmodels.api         as sm
import statsmodels.formula.api as smf
import os

# This statement allow to display plot without asking to 
%matplotlib inline

# always make it pretty 
plt.style.use('ggplot')

## Diving into the data

In [3]:
df = pd.read_csv('Allstate-cost-cleaned.csv',
    dtype = { # indicate categorical variables
        'A': 'category',
        'B': 'category',
        'C': 'category',
        'D': 'category',
        'E': 'category',
        'F': 'category',
        'G': 'category',
        'car_value': 'category',
        'state': 'category'
    }
)

The following are the columns in the dataset:

1. **state**: State where shopping point occurred
2. **group_size**: How many people will be covered under the policy (1, 2, 3 or 4) 
3. **homeowner**: Whether the customer owns a home (0=no, 1=yes)
4. **car_age**: Age of the customer's car (How old the car is)
5. **car_value**: Value of the car when it was new
6. **risk_factor**: An ordinal assessment of how risky the customer is (0,1, 2, 3, 4) 
7. **age_oldest**: Age of the oldest person in customer's group
8. **age_youngest**: Age of the youngest person in customer's group
9. **married_couple**: Does the customer group contain a married couple (0=no, 1=yes) 
10. **C_previous**: What the customer formerly had or currently has for product option C (0=nothing, 1, 2, 3,4)
11. **duration_previous**: How long (in years) the customer was covered by their previous issuer
12. **A,B,C,D,E,F,G**: The coverage options:
13. **A**: Collision (levels: 0, 1, 2);
14. **B**: Towing (levels: 0, 1);
15. **C**: Bodily Injury (BI, levels: 1, 2, 3, 4);
16. **D**: Property Damage (PD, levels 1, 2, 3);
17. **E**: Rental Reimbursement (RR, levels: 0, 1);
18. **F**: Comprehensive (Comp, levels: 0, 1, 2, 3);
19. **G**: Medical/Personal Injury Protection (Med/PIP, levels: 1, 2, 3, 4)
20. **cost**: cost of the quoted coverage options 

In [4]:
df.head(10)

,Unnamed: 0,state,group_size,homeowner,car_age,car_value,risk_factor,age_oldest,age_youngest,married_couple,C_previous,duration_previous,A,B,C,D,E,F,G,cost
0,0,OK,1,0,9,f,0.0,24,24,0,3.0,9.0,0,0,1,1,0,0,4,543
1,1,OK,1,0,9,f,0.0,24,24,0,3.0,9.0,2,1,1,3,1,3,2,611
2,2,PA,1,1,7,f,0.0,74,74,0,2.0,15.0,2,0,2,3,1,2,2,691
3,3,PA,1,1,7,f,0.0,74,74,0,2.0,15.0,2,0,2,3,1,2,2,695
4,4,AR,1,0,4,d,4.0,26,26,0,3.0,1.0,1,0,1,1,0,2,2,628
5,5,AR,1,0,4,d,4.0,26,26,0,3.0,1.0,1,0,2,1,0,2,2,625
6,6,AR,1,0,4,d,4.0,26,26,0,3.0,1.0,1,0,2,1,0,2,2,628
7,7,OK,1,0,13,f,3.0,22,22,0,0.0,0.0,0,0,1,1,0,0,2,596
8,8,OK,1,0,13,f,3.0,22,22,0,0.0,0.0,2,0,1,1,0,3,2,711
9,9,OK,1,0,13,f,3.0,22,22,0,0.0,0.0,2,0,1,1,0,3,2,722


In [5]:
df = df.drop('Unnamed: 0', axis=1)

### Exercise 1:

Write code to visualize the relationship between cost and the following variables. Choose your plots judiciously based on what you know about each variable. Different variable types (categorical vs. numerical) should have different types of plots (e.g. scatter, boxplot, violin plot, etc.) Group your plots together using the `plt.subplot()` function.

1. `car_age`
2. `age_oldest`
3. `age_youngest`
4. `duration_previous`
5. `C_previous`
6. `homeowner`
7. `group_size`
8. `car_age`
9. Categories A-G (7 different plots)

**Answer.**

-------

### Exercise 2:

Convert all categorical data to be in the one-hot encoding format.

**Answer.**

In [6]:
df.dtypes

state                category
group_size              int64
homeowner               int64
car_age                 int64
car_value            category
risk_factor           float64
age_oldest              int64
age_youngest            int64
married_couple          int64
C_previous            float64
duration_previous     float64
A                    category
B                    category
C                    category
D                    category
E                    category
F                    category
G                    category
cost                    int64
dtype: object

In [7]:
df_encoded = pd.get_dummies(df,columns=['car_value', 'state','A','B','C','D','E', 'F','G'], drop_first=True).copy()
df_encoded

,group_size,homeowner,car_age,risk_factor,age_oldest,age_youngest,married_couple,C_previous,duration_previous,cost,...,C_4,D_2,D_3,E_1,F_1,F_2,F_3,G_2,G_3,G_4
0,1,0,9,0.0,24,24,0,3.0,9.0,543,...,0,0,0,0,0,0,0,0,0,1
1,1,0,9,0.0,24,24,0,3.0,9.0,611,...,0,0,1,1,0,0,1,1,0,0
2,1,1,7,0.0,74,74,0,2.0,15.0,691,...,0,0,1,1,0,1,0,1,0,0
3,1,1,7,0.0,74,74,0,2.0,15.0,695,...,0,0,1,1,0,1,0,1,0,0
4,1,0,4,4.0,26,26,0,3.0,1.0,628,...,0,0,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15478,1,0,2,1.0,70,70,0,4.0,9.0,643,...,1,0,1,1,0,1,0,0,1,0
15479,1,0,2,1.0,70,70,0,4.0,9.0,643,...,1,0,1,1,0,1,0,0,1,0
15480,1,0,2,1.0,70,70,0,4.0,9.0,647,...,1,0,1,1,0,1,0,0,1,0
15481,1,1,0,3.0,25,25,0,2.0,6.0,642,...,0,0,1,0,0,0,0,1,0,0


-------

## Fitting a multiple linear regression

### Exercise 3:

Split your data into training and testing sets (an 80-20 split is a good starting point).

**Note:** Keep random seed as 1337 in the code cell

**Answer.**

In [8]:
np.random.seed(1337) 

ndata = len(df)

idx_train = np.random.choice(range(ndata),int(0.8*ndata),replace=False)

idx_test  = np.asarray(list(set(range(ndata)) - set(idx_train)))
train     = df.iloc[idx_train] # the training data set
test      = df.iloc[idx_test]  # the test data set
print(train.shape) 
print(test.shape)  

(12386, 19)
(3097, 19)


-------

### Exercise 4:

#### 4.1

Fit a multiple linear regression model to the training data regressing cost against all the other variables. Call this `model_all`. What is the AIC value?

**Answer.**

In [9]:
formula_all = "cost ~ "

for i in df_encoded.columns:
    if i in ('cost' ):
        pass
    else:
        formula_all += f" {i} +"
        
formula_all = formula_all[:-2]
formula_all

'cost ~  group_size + homeowner + car_age + risk_factor + age_oldest + age_youngest + married_couple + C_previous + duration_previous + car_value_b + car_value_c + car_value_d + car_value_e + car_value_f + car_value_g + car_value_h + car_value_i + state_AR + state_CO + state_CT + state_DC + state_DE + state_FL + state_GA + state_IA + state_ID + state_IN + state_KS + state_KY + state_MD + state_ME + state_MO + state_MS + state_MT + state_ND + state_NE + state_NH + state_NM + state_NV + state_NY + state_OH + state_OK + state_OR + state_PA + state_RI + state_SD + state_TN + state_UT + state_WA + state_WI + state_WV + state_WY + A_1 + A_2 + B_1 + C_2 + C_3 + C_4 + D_2 + D_3 + E_1 + F_1 + F_2 + F_3 + G_2 + G_3 + G_4'

In [10]:
model_all = smf.ols(formula=formula_all, data= df_encoded).fit()

print(model_all.summary())

                            OLS Regression Results                            
Dep. Variable:                   cost   R-squared:                       0.438
Model:                            OLS   Adj. R-squared:                  0.436
Method:                 Least Squares   F-statistic:                     179.4
Date:                Mon, 02 Aug 2021   Prob (F-statistic):               0.00
Time:                        02:21:05   Log-Likelihood:                -77228.
No. Observations:               15483   AIC:                         1.546e+05
Df Residuals:                   15415   BIC:                         1.551e+05
Df Model:                          67                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept           675.4716      4.98

-------

#### 4.2 

According to `model_all`, which states are most and least expensive?

**Answer.**

In [11]:
print(model_all.params.sort_values(ascending=False)[0:10], '\n')

model_all.params.sort_values(ascending=False)[58:68]


Intercept    675.471601
state_NY      39.751177
state_DE      37.177927
state_DC      36.780392
A_2           32.725870
state_CT      31.601078
A_1           27.549317
state_WV      27.202818
state_RI      26.884047
state_NV      23.701646
dtype: float64 



car_value_h   -27.862960
state_WI      -31.640965
state_ME      -33.590503
car_value_g   -35.573458
car_value_d   -39.590594
car_value_e   -39.616808
car_value_f   -39.849312
car_value_c   -45.574971
state_IA      -50.582827
car_value_b   -56.977730
dtype: float64

The most expensive car insurance are in  New York (NY), Delaware (DE) and District of Columbia (DC) States

The least expensive car insurance are in Iowa(IA), Maine(ME) and Wisconsin(WI) states.

-------

#### 4.3

Interpret the coefficients of `group_size`, `homeowner`, `car_age`, `risk_factor`, `age_oldest`, `age_youngest`       `married_couple` , `duration_previous`. Do the signs and values of these coefficients make sense to you in the context of this business problem?

**Answer.**

In [12]:
model_all.params[['group_size', 
                  'homeowner', 
                  'car_age', 
                  'age_oldest', 
                  'age_youngest', 
                  'married_couple', 
                  'duration_previous'
                 ]]

group_size            3.113752
homeowner           -14.367412
car_age              -0.729927
age_oldest            0.589880
age_youngest         -0.985884
married_couple       -9.821222
duration_previous    -1.508108
dtype: float64

The coefficients and their sign are almost all expected, but car_age is unexpected because in terms of insurance we think and hear that if the car model is old the insurance increases but for this model it does not apply. Also a curious fact is that if we are homeowners the insurance decreases a considerable amount.

-------

### Exercise 5:

Which variables from `model_all` are statistically significant? (For categorical variables, consider them to be significant if at least one of their categories are statistically significant). Refit the model using only these variables; call this `model_sig`. How does this model compare to the previous model?

**Answer.**

In [17]:
sign_variables = model_all.pvalues[(model_all.pvalues > 0.05/len(model_all.params))]
print(sign_variables)



group_size     0.018665
risk_factor    0.002773
car_value_i    0.320186
state_AR       0.619939
state_KS       0.061570
state_MS       0.636833
state_MT       0.035332
state_ND       0.016023
state_NE       0.008343
state_NM       0.635879
state_OR       0.005005
state_SD       0.162752
state_WA       0.038341
state_WY       0.443805
B_1            0.004076
C_2            0.548479
C_3            0.363510
C_4            0.038018
D_2            0.305683
D_3            0.238153
G_3            0.373691
dtype: float64


In [24]:
formula_sign = "cost ~ "
for i in sign_variables.index:
    formula_sign += f" {i} +"
    
formula_sign = formula_sign[:-2]

model_sign = smf.ols(formula=formula_sign, data= df_encoded).fit()

print(model_sign.summary())

                            OLS Regression Results                            
Dep. Variable:                   cost   R-squared:                       0.036
Model:                            OLS   Adj. R-squared:                  0.034
Method:                 Least Squares   F-statistic:                     27.20
Date:                Mon, 02 Aug 2021   Prob (F-statistic):          2.31e-105
Time:                        02:42:18   Log-Likelihood:                -81411.
No. Observations:               15483   AIC:                         1.629e+05
Df Residuals:                   15461   BIC:                         1.630e+05
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept     629.8968      1.429    440.744      

-------

### Exercise 6:

In addition to the variables in `model_sig`, add terms for:

1. square of `age_youngest`
2. square term for the age of the car
3. interaction term for `car_value` and `age_youngest`

and save it to a new model `model_sig_plus`.

**Answer.**

In [28]:
formula_sign_plus


'cost ~  group_size + risk_factor + car_value_i + state_AR + state_KS + state_MS + state_MT + state_ND + state_NE + state_NM + state_OR + state_SD + state_WA + state_WY + B_1 + C_2 + C_3 + C_4 + D_2 + D_3 + G_3+  I(age_youngest**2) + I(car_age**2)'

In [29]:
formula_sign_plus = formula_sign + "+  I(age_youngest**2) + I(car_age**2) + car_value_i*age_youngest"

model_sign_plus = smf.ols(formula=formula_sign_plus, data= df_encoded).fit()

print(model_sign_plus.summary())

                            OLS Regression Results                            
Dep. Variable:                   cost   R-squared:                       0.127
Model:                            OLS   Adj. R-squared:                  0.126
Method:                 Least Squares   F-statistic:                     90.19
Date:                Mon, 02 Aug 2021   Prob (F-statistic):               0.00
Time:                        02:47:32   Log-Likelihood:                -80637.
No. Observations:               15483   AIC:                         1.613e+05
Df Residuals:                   15457   BIC:                         1.615e+05
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept               

-------

## Feature selection

To reduce the number of features, it can often be helpful to aggregate the categories; for example, we can create a new variable by assigning each state to a larger region:

In [16]:
state_regions = pd.read_csv('https://raw.githubusercontent.com/cphalpert/census-regions/master/us%20census%20bureau%20regions%20and%20divisions.csv')
# should download the above file
state_regions

,State,State Code,Region,Division
0,Alaska,AK,West,Pacific
1,Alabama,AL,South,East South Central
2,Arkansas,AR,South,West South Central
3,Arizona,AZ,West,Mountain
4,California,CA,West,Pacific
5,Colorado,CO,West,Mountain
6,Connecticut,CT,Northeast,New England
7,District of Columbia,DC,South,South Atlantic
8,Delaware,DE,South,South Atlantic
9,Florida,FL,South,South Atlantic


### Exercise 7:

#### 7.1

Create a new column where a state is replaced with the region it is in according to the above table.

**Answer.**

-------

#### 7.2

Fit the model as in `model_sig_plus` but this time use `region` instead of `state`. Call this `model_region`.

**Answer.**

-------

### Exercise 8:

#### 8.1

What should we do next to minimize features?

**Answer.**

-------

#### 8.2

Using a method of your choice, find the numerical feature(s) in `model_region`, except for the three we added in Exercise 6, which exhibit multicollinearity. **Hint:** consider looking at correlations.

**Answer.**

-------

#### 8.3:

Refit `model_region` after dropping these redundant predictor(s); call this `model_region_no_oldest`.

**Answer.**

-------

#### 8.4

What would you do to diagnose the `model_region_no_oldest` fit? What does this diagnosis suggest to you? (Hint: try plotting the residuals in various ways.)

**Answer.**

-------

### Exercise 9:

#### 9.1

Find the best Box-Cox transformation of `cost` used to fit `model_region_no_oldest`. What value do you get?

**Answer.**

-------

#### 9.2

Refit `model_region_no_oldest`, but now with the transformation as suggested by the Box-Cox. Call it `model_region_no_oldest_box_cox`.

**Answer.**

-------

## Conclusion

In this, you practiced creating linear models using `statsmodels` and iteratively trimming the input variables to go from including all the variables in the dataset to using only the most relevant variables. You excluded those variables that were statistically insignificant and removed those that had high correlation. Finally, we performed some feature engineering in an attempt to remove some tail behavior that deviates from the normal distribution to better fit our linear model. In the end, we had a very minimal model that contained variables that other insurance companies use to charge premiums that gave us insight on how we can better serve a niche population. 